In [136]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, GenerationConfig
import torch
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory,ConversationSummaryMemory, ConversationBufferWindowMemory
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser, CommaSeparatedListOutputParser

In [2]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
pipe = pipeline("text-generation", 
                model=model, 
                tokenizer=tokenizer,
                device=0, 
                max_new_tokens=2000,
                do_sample=True, 
                top_k=20, 
                top_p=0.7,
                early_stopping=True,
                num_beams=2
               )
hf = HuggingFacePipeline(pipeline=pipe)

In [91]:
question_schema = ResponseSchema(name='Question', description="Question Generated on the given topic")
choice1_schema = ResponseSchema(name='Choice1', description='Choice 1 for the given question')
choice2_schema = ResponseSchema(name='Choice2', description='Choice 2 for the given question')
choice3_schema = ResponseSchema(name='Choice3', description='Choice 3 for the given question')
choice4_schema = ResponseSchema(name='Choice4', description='Choice 4 for the given question')
answer_schema = ResponseSchema(name='Answer', description='One of the selected choices out of 4 choices given as the answer. Eg Choice1')
explanation_schema = ResponseSchema(name='Explanation', description = 'Explanation why a particular choice is selected as the answer')

response_schemas = [question_schema, 
                    choice1_schema,
                    choice2_schema,
                    choice3_schema,
                    choice4_schema,
                    answer_schema,
                    explanation_schema
                   ]

In [92]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
# focused on the topic of {subject}
#{chat_history}

In [153]:
template = """You are a KnowledgeStore which generates MCQ questions of desired difficulty on a given topic.

{chat_history}

Human: Generate a MCQ type question for graduate students of {difficulty} difficulty level focussed on {subject}. 
The question should present a significant challenge, aligning with master's level coursework, and should ideally incorporate real-world applications or data to contextualize the mathematical or statistical concepts involved. 
Ensure the question demand a deep understanding and application of theoretical principles, possibly involving multiple steps or the integration of several concepts.
Ensure that Question is strictly multiple choice question with exactly 4 choices.
Choice1, Choice2, Choice3, Choice4
The answer should definitely be one of the Choices.

{format_instructions}
"""

#
prompt = PromptTemplate.from_template(template)
prompt = PromptTemplate(
    input_variables=["chat_history", 
                     "difficulty",
                     "subject",
                     "format_instructions"], template=template
)
#memory = ConversationSummaryMemory(llm=hf, input_key='subject')
memory = ConversationBufferWindowMemory(k=5, memory_key="chat_history", input_key='subject')
#memory = ConversationBufferMemory(memory_key="chat_history", input_key='subject')

In [154]:
#memory.load_memory_variables({})

In [157]:
llm_chain = LLMChain(
    llm=hf,
    prompt=prompt,
    memory=memory,
    verbose=True
)

In [158]:
out = llm_chain.run({'difficulty' : 'easy',
                     'subject' : 'history of USA',
                     'format_instructions' : format_instructions
                    })

/home/aagarwal/miniconda3/envs/OpenShape/lib/python3.9/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:
You are a KnowledgeStore which generates MCQ questions of desired difficulty on a given topic.



Human: Generate a MCQ type question for graduate students of easy difficulty level focussed on history of USA. 
The question should present a significant challenge, aligning with master's level coursework, and should ideally incorporate real-world applications or data to contextualize the mathematical or statistical concepts involved. 
Ensure the question demand a deep understanding and application of theoretical principles, possibly involving multiple steps or the integration of several concepts.
Ensure that Question is strictly multiple choice question with exactly 4 choices.
Choice1, Choice2, Choice3, Choice4
The answer should definitely be one of the Choices.

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"Question": string  /

In [159]:
output_parser.parse(out)

{'Question': 'In what year did the first transcontinental railroad in the United States of America get completed?',
 'Choice1': '1860',
 'Choice2': '1865',
 'Choice3': '1870',
 'Choice4': '1875',
 'Answer': 'Choice2',
 'Explanation': 'The first transcontinental railroad in the United States of America was completed in 1865. The Central Pacific Railroad and the Union Pacific Railroad joined their tracks in Promontory Summit, Utah, on May 10, 1869. However, the last spike was not driven until May 10, 1869. Therefore, the correct answer is 1865.'}

In [75]:
output_parser.parse(out)

{'Question': 'Which event marked the turning point in the American Revolution?',
 'Choice1': 'The signing of the Declaration of Independence',
 'Choice2': 'The Battle of Saratoga',
 'Choice3': 'The Boston Tea Party',
 'Choice4': 'The Battle of Bunker Hill',
 'Answer': 'The Battle of Saratoga',
 'Explanation': 'The Battle of Saratoga, fought in 1777, was a significant turning point in the American Revolution. It was the first major American victory and boosted morale and international support for the American cause.'}

In [74]:
#out = llm_chain.run({'human_input' : 'Question on Indian History', 'format_instructions' : format_instructions})

In [71]:
output_parser.parse(out)

{'Question': 'Which event marked the beginning of the American Revolution?',
 'Choice1': 'The Boston Tea Party',
 'Choice2': 'The signing of the Declaration of Independence',
 'Choice3': 'The Battle of Bunker Hill',
 'Choice4': 'The Stamp Act',
 'Answer': 'The Boston Tea Party',
 'Explanation': "The Boston Tea Party was a significant political protest by the American colonists in response to the British government's imposition of a tax on tea without the consent of the colonists. This event is considered the beginning of the American Revolution."}

In [64]:
output_parser.parse(out)

{'Question': "Which Sultan and in which period established the Delhi Sultanate's first university, modeling it after Al-Jamiat al-Qarawiyyin of Fez, a significant center for the translation and dissemination of Islamic and Persian knowledge in India?",
 'Choice1': 'Sultan Alauddin Khilji (1296-1316)',
 'Choice2': 'Sultan Muhammad bin Tughlaq (1325-1351)',
 'Choice3': 'Sultan Iltutmish (1211-1236)',
 'Choice4': 'Sultan Qutb-ud-din Aibak (1206-1210)',
 'Answer': 'Choice2',
 'Explanation': "Sultan Muhammad bin Tughlaq established the Delhi Sultanate's first university in the early 14th century, modeling it after Al-Jamiat al-Qarawiyyin of Fez. This university played a crucial role in the dissemination of Islamic and Persian knowledge in India during the medieval period."}

In [62]:
output_parser.parse(out)

{'Question': 'Which Sultan initiated the construction of the Qutub Minar in Indian history and in which period did this occur? Additionally, explain the significance of the Qutub Minar in the context of Islamic architecture and its influence on future structures.',
 'Choice1': 'Mughal Emperor Akbar (1556-1605)',
 'Choice2': 'Mughal Emperor Shah Jahan (1628-1658)',
 'Choice3': 'Sultan Qutb-ud-din Aibak (1206-1210)',
 'Choice4': 'Sultan Alauddin Khilji (1296-1316)',
 'Answer': 'Choice3',
 'Explanation': "Sultan Qutb-ud-din Aibak initiated the construction of the Qutub Minar in the late 12th century during the Slave Dynasty. The Qutub Minar is a significant example of early Islamic architecture in India and is considered a precursor to the Indo-Islamic architectural style. It is a masterpiece of brickwork, showcasing intricate carvings and calligraphy. The minaret's influence can be seen in later structures such as the Tomb of Iltutmish and the Alai Darwaza."}

In [11]:
llm_chain.predict(human_input="Easy difficulty level question on Indian History")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


" I'm sorry for the misunderstanding earlier. Based on your requirements, here's an example of an easy question on Indian History for graduate students:\n\nQuestion: Which Indian ruler, known for his patronage of arts and sciences, is believed to have commissioned the construction of the Qutub Minar in Delhi?\n\nA) Akbar\nB) Shah Jahan\nC) Muhammad bin Tughlaq\nD) Qutub-ud-din Aibak\n\nAnswer: D) Qutub-ud-din Aibak\n\nThis question tests the student's understanding of Indian history and requires them to recall specific details about a historical figure and their contributions. It is a multiple choice question, and the answer is one of the choices provided."

In [12]:
llm_chain.predict(human_input="Easy difficulty level question on Indian History")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


" I'm an AI designed to generate questions based on a given topic and difficulty level. However, your request seems to have some contradictions. You've asked for an easy question, but then you've specified that it should align with master's level coursework, involve real-world applications, and require a deep understanding and application of theoretical principles. These requirements are typically associated with more complex, challenging questions.\n\nFor an easy question on Indian History for graduate students, I would suggest the following:\n\nQuestion: Which Indian ruler, known for his patronage of arts and sciences, is believed to have commissioned the construction of the Qutub Minar in Delhi?\n\nA) Akbar\nB) Shah Jahan\nC) Muhammad bin Tughlaq\nD) Qutub-ud-din Aibak\n\nAnswer: D) Qutub-ud-din Aibak\n\nThis question tests the student's understanding of Indian history and requires them to recall specific details about a historical figure and their contributions. It is a multiple ch

In [45]:
input_dict = {"difficulty": 'easy',
                "subject": 'History of India',
                "format_instructions" : format_instructions}

llm_chain.invoke(input_dict)

/home/aagarwal/miniconda3/envs/OpenShape/lib/python3.9/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ValueError: One input key expected got ['subject', 'format_instructions', 'difficulty']